In [38]:
# ============================================================
# 006_RAG-Based Research Question Generator
# ============================================================
#
# Overview
# ----------------
# This notebook implements a Retrieval-Augmented Generation (RAG) workflow
# to generate *researchable* questions grounded in your existing materials.
#
# Unlike summarization-only pipelines, the goal here is to turn a broad
# research idea into a shortlist of concrete, empirically testable RQs by:
#   1) retrieving relevant context from your knowledge base, then
#   2) generating structured RQ drafts conditioned on that evidence.
#
# In this project setup, the knowledge base is built from:
# - Google Drive: PDFs (academic papers, reports)
# - Notion: prior RQ memos and notes
# and retrieval is performed via an OpenAI Vector Store.
#
# The final outputs are aligned to a Notion database schema, enabling a
# "generate → review → refine → store" loop inside a personal Research OS.
#
#
# Inputs / Outputs
# ----------------
# Inputs:
# - Google Drive folder (PDF corpus) identified by DRIVE_FOLDER_ID
# - Notion database / data source (RQ memos) via NOTION_TOKEN and NOTION_RQ_DB_ID
# - OpenAI API key for vector store retrieval and LLM generation
# - Prompt templates and runtime parameters (top-k, temperature, etc.)
#
# Outputs:
# - RQ candidates in a DB-aligned schema:
#     - Name (Japanese research question)
#     - Rationale / Background (JP)
#     - Gap Identified (JP)
#     - Proposed Approach (JP)
#     - Priority (fixed: Medium)
#     - Status (fixed: New)
#     - Tags (English keywords, ~5)
# - Evidence references used for each RQ (e.g., evidence_used: [1, 2, ...])
# - Exportable artifacts (CSV / JSON) and an execution manifest for traceability
# - Optionally: newly created pages in the Notion RQ database
#
#
# Structure
# ----------------
# Cell 0 : Purpose, scope, and design assumptions
# Cell 1 : Imports and global configuration (OpenAI / Notion / Google)
# Cell 2 : Load source materials (Google Drive PDFs + Notion RQ notes)
# Cell 3 : Embedding setup and vector store initialization (OpenAI Vector Store)
# Cell 4 : Retrieval logic (top-k semantic search over the vector store)
# Cell 5 : Prompt construction (DB-aligned JSON schema + Japanese output constraints)
# Cell 6 : Interactive generation UI (retrieve → generate → Go/No-Go → edit)
# Cell 7 : Post-processing (normalization, filtering, deduplication; DB-aligned)
# Cell 8 : Export results and quick inspection (CSV/JSON + run manifest)
# Cell 9 : Write selected RQs to Notion (DB-aligned, schema + option checks)
#
#
# Notes
# ----------------
# - This notebook generates questions; it does not attempt to answer them.
# - Retrieval quality determines generation quality. If outputs look noisy,
#   inspect the retrieved contexts (Cell 4/6) and adjust top-k or corpus inputs.
# - Treat generated RQs as drafts. Use the Go/No-Go + manual edit step to
#   improve clarity, feasibility, and empirical specificity.
# - The pipeline is modular: you can swap retrieval backends, prompts, models,
#   or output schema without rewriting the entire workflow.
# - If Notion writes fail, the most common causes are:
#   (1) the DB property types do not match, or
#   (2) required select/status options (e.g., Medium/New) are missing.
# ============================================================


In [21]:
# ============================================================
# Cell 1 : Imports and Global Configuration
# ============================================================
#
# This cell defines all core imports and global configuration variables
# used throughout the notebook.
#
# It also:
# - loads environment variables from env.txt
# - validates API credentials (OpenAI / Notion / Google Drive)
# - prepares reusable service clients (OpenAI, Notion headers, Drive service factory)
#
# All environment-dependent settings should be modified here.
#

# ----------------
# Standard Library
# ----------------
import os
import json
import pathlib
from typing import List, Dict, Any, Optional

# ----------------
# Data Handling
# ----------------
import pandas as pd
import numpy as np

# ----------------
# Environment Variables
# ----------------
from dotenv import load_dotenv

# ----------------
# HTTP
# ----------------
import requests

# ----------------
# Google API (Drive)
# ----------------
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow

# ----------------
# OpenAI
# ----------------
from openai import OpenAI


# ------------------------------------------------------------
# Load environment variables
# ------------------------------------------------------------
# Explicitly load env.txt (instead of default .env)
load_dotenv("env.txt")


# ============================================================
# Global Paths
# ============================================================
BASE_DIR = pathlib.Path(".")
DATA_DIR = BASE_DIR / "data"
OUTPUT_DIR = BASE_DIR / "outputs"

DATA_DIR.mkdir(exist_ok=True)
OUTPUT_DIR.mkdir(exist_ok=True)


# ============================================================
# Utility: required env var checker
# ============================================================
def require_env(name: str) -> str:
    """
    Raise a clear error if a required environment variable is missing.
    """
    val = os.getenv(name)
    if not val:
        raise ValueError(f"{name} is missing in env.txt")
    return val


# ============================================================
# OpenAI configuration
# ============================================================
OPENAI_API_KEY = require_env("OPENAI_API_KEY")
print("✅ OPENAI_API_KEY loaded successfully")

openai_client = OpenAI(api_key=OPENAI_API_KEY)


# ============================================================
# Notion configuration
# ============================================================
NOTION_TOKEN = require_env("NOTION_TOKEN")
NOTION_VERSION = require_env("NOTION_VERSION")            # e.g., "2022-06-28" or your pinned version
NOTION_RQ_DB_ID = require_env("NOTION_RQ_DB_ID")          # Research Question database ID
# Optional (only if you use it elsewhere)
NOTION_RQ_DATA_SOURCE_ID = os.getenv("NOTION_RQ_DATA_SOURCE_ID")

NOTION_HEADERS = {
    "Authorization": f"Bearer {NOTION_TOKEN}",
    "Notion-Version": NOTION_VERSION,
    "Content-Type": "application/json",
}

# Quick auth check (safe: does not reveal token)
try:
    r = requests.get("https://api.notion.com/v1/users/me", headers=NOTION_HEADERS, timeout=30)
    if r.status_code == 200:
        print("✅ Notion auth OK")
    else:
        print("⚠️ Notion auth check failed:", r.status_code, r.text[:200])
except Exception as e:
    print("⚠️ Notion auth check error:", type(e).__name__, str(e))


# ============================================================
# Google Drive configuration
# ============================================================

# Read-only scope is sufficient for paper ingestion
DRIVE_SCOPES = ["https://www.googleapis.com/auth/drive.readonly"]

# ------------------------------------------------------------
# OAuth client secret handling
# ------------------------------------------------------------
# Priority:
# 1) GOOGLE_OAUTH_CLIENT_SECRET_JSON in env.txt
# 2) Default local filename (checked in working directory)
#
DEFAULT_GOOGLE_CLIENT_SECRET = (
    "client_secret_750875982200-85rnsoqhr2af2b13peueev0bm60q22sh.apps.googleusercontent.com.json"
)

GOOGLE_OAUTH_CLIENT_SECRET_JSON = os.getenv("GOOGLE_OAUTH_CLIENT_SECRET_JSON")

if GOOGLE_OAUTH_CLIENT_SECRET_JSON:
    client_secret_path = pathlib.Path(GOOGLE_OAUTH_CLIENT_SECRET_JSON)
else:
    client_secret_path = pathlib.Path(DEFAULT_GOOGLE_CLIENT_SECRET)

if not client_secret_path.exists():
    raise ValueError(
        "Google OAuth client secret JSON not found.\n"
        "Either:\n"
        "  - set GOOGLE_OAUTH_CLIENT_SECRET_JSON in env.txt, or\n"
        f"  - place the file at: {DEFAULT_GOOGLE_CLIENT_SECRET}"
    )

print(f"✅ Google OAuth client secret loaded: {client_secret_path}")

# ------------------------------------------------------------
# Token cache path (still configurable via env.txt)
# ------------------------------------------------------------
GOOGLE_TOKEN_JSON = os.getenv("GOOGLE_TOKEN_JSON", "google_token.json")
DRIVE_FOLDER_ID = require_env("DRIVE_FOLDER_ID")

def get_drive_service():
    """
    Authenticate with Google Drive API and return a Drive service object.
    Uses OAuth with token caching to avoid repeated authentication.
    """
    creds = None
    token_path = pathlib.Path(GOOGLE_TOKEN_JSON)

    if token_path.exists():
        creds = Credentials.from_authorized_user_file(
            str(token_path), DRIVE_SCOPES
        )

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                str(client_secret_path), DRIVE_SCOPES
            )
            # Local environment: browser-based auth
            creds = flow.run_local_server(port=0)

        token_path.write_text(creds.to_json(), encoding="utf-8")

    return build("drive", "v3", credentials=creds)

# Initialize Drive client
try:
    drive = get_drive_service()
    print("✅ Google Drive client initialized")
except Exception as e:
    drive = None
    print("⚠️ Google Drive init error:", type(e).__name__, str(e))


# ============================================================
# Retrieval / Generation Parameters
# ============================================================
TOP_K_RETRIEVAL = 5      # Number of documents retrieved per query
MIN_SIMILARITY = 0.2    # Optional similarity threshold

EMBEDDING_MODEL = "text-embedding-3-small"
LLM_MODEL = "gpt-4.1-mini"
MAX_TOKENS = 2000
TEMPERATURE = 0.7


# ============================================================
# Similarity Utility (NumPy only; avoids sklearn/pyarrow import issues)
# ============================================================
def cosine_similarity_matrix(A: np.ndarray, B: np.ndarray) -> np.ndarray:
    """
    Compute cosine similarity between two 2D arrays:
      A: (n, d)
      B: (m, d)
    Returns:
      (n, m) cosine similarity matrix
    """
    A = A.astype(np.float32)
    B = B.astype(np.float32)
    A = A / (np.linalg.norm(A, axis=1, keepdims=True) + 1e-12)
    B = B / (np.linalg.norm(B, axis=1, keepdims=True) + 1e-12)
    return A @ B.T


# ============================================================
# Reproducibility & Display
# ============================================================
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

pd.set_option("display.max_colwidth", 120)
pd.set_option("display.max_rows", 50)


✅ OPENAI_API_KEY loaded successfully
✅ Notion auth OK
✅ Google OAuth client secret loaded: client_secret_750875982200-85rnsoqhr2af2b13peueev0bm60q22sh.apps.googleusercontent.com.json
✅ Google Drive client initialized


In [8]:
# ============================================================
# Cell 2 : Ingest Sources (Google Drive PDFs + Notion RQs)
# ============================================================
#
# This cell "bulk-ingests" two sources into local artifacts for RAG:
# 1) Google Drive folder: downloads PDF papers into outputs/drive_pdfs/
# 2) Notion RQ database: exports each row/page into outputs/notion_rqs/*.txt
#
# Output artifacts are designed to be:
# - traceable (stable filenames / ids)
# - incrementally updatable (skip if already downloaded/exported)
# - vector-store friendly (ready to upload as files)
#
# Requirements (from Cell 1):
# - drive: authenticated Google Drive service object
# - DRIVE_FOLDER_ID
# - NOTION_HEADERS, NOTION_RQ_DB_ID
# - OUTPUT_DIR
#

import io
import time
from pathlib import Path
from tqdm import tqdm
import requests

# ----------------------------
# Output directories
# ----------------------------
DRIVE_DOWNLOAD_DIR = OUTPUT_DIR / "drive_pdfs"
NOTION_EXPORT_DIR = OUTPUT_DIR / "notion_rqs"
DRIVE_DOWNLOAD_DIR.mkdir(parents=True, exist_ok=True)
NOTION_EXPORT_DIR.mkdir(parents=True, exist_ok=True)

# ----------------------------
# Google Drive: list + download
# ----------------------------
def list_all_pdfs_in_folder(folder_id: str, page_size: int = 200):
    """
    List all PDF files in a Google Drive folder (pagination).
    Returns: list of dicts with keys like id, name, modifiedTime, size.
    """
    if drive is None:
        raise RuntimeError("Google Drive client is not initialized (drive is None).")

    q = (
        f"'{folder_id}' in parents "
        f"and mimeType='application/pdf' "
        f"and trashed=false"
    )

    files = []
    page_token = None
    while True:
        resp = drive.files().list(
            q=q,
            pageSize=page_size,
            pageToken=page_token,
            fields="nextPageToken, files(id,name,modifiedTime,size)"
        ).execute()

        files.extend(resp.get("files", []))
        page_token = resp.get("nextPageToken")
        if not page_token:
            break

    return files

def download_drive_pdf(file_id: str, dest_path: Path):
    """
    Download a Drive file (PDF) to dest_path.
    """
    request = drive.files().get_media(fileId=file_id)
    bio = io.BytesIO()
    downloader = MediaIoBaseDownload(bio, request)

    done = False
    while not done:
        status, done = downloader.next_chunk()

    dest_path.write_bytes(bio.getvalue())

def sanitize_filename(name: str) -> str:
    """
    Minimal filename sanitizer (safe for macOS/Linux/Windows).
    """
    bad = ['/', '\\', ':', '*', '?', '"', '<', '>', '|']
    for ch in bad:
        name = name.replace(ch, "_")
    return name.strip()

# ---- Drive run
drive_pdf_files = list_all_pdfs_in_folder(DRIVE_FOLDER_ID)
print(f"✅ Google Drive: found {len(drive_pdf_files)} PDFs in folder.")

drive_downloaded_paths = []
drive_skipped = 0
drive_downloaded = 0

for f in tqdm(drive_pdf_files):
    file_id = f["id"]
    name = sanitize_filename(f["name"])
    out_path = DRIVE_DOWNLOAD_DIR / name

    # Incremental behavior: skip if already exists and non-empty
    if out_path.exists() and out_path.stat().st_size > 0:
        drive_skipped += 1
        drive_downloaded_paths.append(str(out_path))
        continue

    download_drive_pdf(file_id, out_path)
    drive_downloaded += 1
    drive_downloaded_paths.append(str(out_path))

print(f"✅ Google Drive: downloaded {drive_downloaded}, skipped {drive_skipped} (already present).")
print(f"📁 PDFs stored at: {DRIVE_DOWNLOAD_DIR}")

# ----------------------------
# Notion: query all pages + export to txt
# ----------------------------
def notion_query_all_pages(database_id: str, page_size: int = 100):
    """
    Query a Notion database and return all pages (pagination).
    """
    url = f"https://api.notion.com/v1/databases/{database_id}/query"
    all_results = []
    payload = {"page_size": page_size}

    has_more = True
    next_cursor = None

    while has_more:
        if next_cursor:
            payload["start_cursor"] = next_cursor

        r = requests.post(url, headers=NOTION_HEADERS, json=payload, timeout=60)
        r.raise_for_status()
        data = r.json()

        all_results.extend(data.get("results", []))
        has_more = data.get("has_more", False)
        next_cursor = data.get("next_cursor")

    return all_results

def extract_plain_text_from_property(prop: dict) -> str:
    """
    Best-effort extraction of human-readable text from common Notion property types.
    """
    t = prop.get("type")
    if t == "title":
        return " ".join([x.get("plain_text", "") for x in prop.get("title", [])]).strip()
    if t == "rich_text":
        return " ".join([x.get("plain_text", "") for x in prop.get("rich_text", [])]).strip()
    if t == "select":
        s = prop.get("select")
        return (s.get("name") if s else "").strip()
    if t == "multi_select":
        return ", ".join([x.get("name", "") for x in prop.get("multi_select", []) if x.get("name")]).strip()
    if t == "number":
        n = prop.get("number")
        return "" if n is None else str(n)
    if t == "status":
        s = prop.get("status")
        return (s.get("name") if s else "").strip()
    if t == "date":
        d = prop.get("date")
        return (d.get("start") if d else "").strip()
    if t == "url":
        return (prop.get("url") or "").strip()
    if t == "people":
        return ", ".join([p.get("name", "") for p in prop.get("people", []) if p.get("name")]).strip()
    return ""

def notion_page_to_text(page: dict) -> str:
    """
    Convert a Notion database row/page into a single text document for RAG.
    """
    props = page.get("properties", {})
    lines = []
    lines.append(f"NOTION_PAGE_ID: {page.get('id','')}")
    lines.append(f"NOTION_URL: {page.get('url','')}")
    lines.append("")

    # Dump properties in a stable order (alphabetical) for determinism
    for key in sorted(props.keys()):
        val = extract_plain_text_from_property(props[key])
        if val:
            lines.append(f"{key}: {val}")

    return "\n".join(lines).strip()

# ---- Notion run
notion_pages = notion_query_all_pages(NOTION_RQ_DB_ID)
print(f"✅ Notion: fetched {len(notion_pages)} pages from RQ database.")

notion_text_paths = []
notion_written = 0
notion_skipped = 0

for p in tqdm(notion_pages):
    # Stable filename based on page_id (strip hyphens)
    page_id = p.get("id", "").replace("-", "")
    out_path = NOTION_EXPORT_DIR / f"rq_{page_id}.txt"

    # Incremental behavior: skip if already exists and non-empty
    if out_path.exists() and out_path.stat().st_size > 0:
        notion_skipped += 1
        notion_text_paths.append(str(out_path))
        continue

    text = notion_page_to_text(p)
    out_path.write_text(text, encoding="utf-8")
    notion_written += 1
    notion_text_paths.append(str(out_path))

print(f"✅ Notion: wrote {notion_written}, skipped {notion_skipped} (already present).")
print(f"📁 Notion RQs stored at: {NOTION_EXPORT_DIR}")

# ----------------------------
# Summary outputs for downstream cells
# ----------------------------
print("\n--- Summary ---")
print("Drive PDF paths:", len(drive_downloaded_paths))
print("Notion txt paths:", len(notion_text_paths))

# These variables are used in the next step (Vector Store upload)
ALL_RAG_FILE_PATHS = drive_downloaded_paths + notion_text_paths
print("Total RAG file paths:", len(ALL_RAG_FILE_PATHS))


✅ Google Drive: found 35 PDFs in folder.


100%|███████████████████████████████████████████| 35/35 [03:30<00:00,  6.02s/it]


✅ Google Drive: downloaded 34, skipped 1 (already present).
📁 PDFs stored at: outputs/drive_pdfs
✅ Notion: fetched 29 pages from RQ database.


100%|█████████████████████████████████████████| 29/29 [00:00<00:00, 2234.41it/s]

✅ Notion: wrote 29, skipped 0 (already present).
📁 Notion RQs stored at: outputs/notion_rqs

--- Summary ---
Drive PDF paths: 35
Notion txt paths: 29
Total RAG file paths: 64


In [9]:
# ============================================================
# Cell 3 : Embedding Setup and Vector Store Initialization
# ============================================================
#
# This notebook uses OpenAI Vector Stores as the retrieval backend.
# When files are added to a vector store, OpenAI automatically:
# - parses file contents,
# - chunks them,
# - builds embeddings,
# - and indexes them for semantic search / file_search usage.
#
# Therefore, we do NOT manually compute embeddings in this notebook.
# Instead, we:
# 1) create (or reuse) a vector store
# 2) upload local artifacts (Drive PDFs + Notion TXT exports) as OpenAI Files
# 3) attach them to the vector store via a file batch
# 4) poll until the vector store is ready for retrieval
#
# Requirements (from previous cells):
# - openai_client (OpenAI client)
# - ALL_RAG_FILE_PATHS (list[str]) created in Cell 2
#
# References:
# - Vector stores API :contentReference[oaicite:1]{index=1}
# - Vector store file batches :contentReference[oaicite:2]{index=2}
# - Vector store files status :contentReference[oaicite:3]{index=3}
# ============================================================

import os
import time
from pathlib import Path
from tqdm import tqdm

# ----------------------------
# Vector Store Configuration
# ----------------------------
VECTOR_STORE_NAME = "RQ-RAG-Drive+Notion"
REUSE_VECTOR_STORE_ID = os.getenv("VECTOR_STORE_ID")  # optional: reuse an existing store if provided

# Upload batching
UPLOAD_PURPOSE = "assistants"  # purpose used for files that will be searched by file_search-style tools

# Polling
POLL_INTERVAL_SEC = 2


# ----------------------------
# Helpers
# ----------------------------
def create_or_reuse_vector_store(name: str, reuse_id: str | None = None):
    """
    Create a new vector store, or reuse an existing one if an ID is provided.
    """
    if reuse_id:
        vs = openai_client.vector_stores.retrieve(reuse_id)
        print(f"✅ Reusing vector store: {vs.id} ({getattr(vs, 'name', '')})")
        return vs

    vs = openai_client.vector_stores.create(name=name)
    print(f"✅ Created vector store: {vs.id} ({name})")
    return vs


def upload_files(file_paths: list[str]) -> list[str]:
    """
    Upload local files to OpenAI Files API and return file_ids.
    """
    file_ids = []
    for p in tqdm(file_paths, desc="Uploading files to OpenAI"):
        path = Path(p)
        if not path.exists():
            raise FileNotFoundError(f"Missing file: {path}")

        with open(path, "rb") as f:
            up = openai_client.files.create(file=f, purpose=UPLOAD_PURPOSE)
        file_ids.append(up.id)

    return file_ids


def attach_files_to_vector_store(vector_store_id: str, file_ids: list[str]):
    """
    Attach uploaded files to a vector store using a file batch.
    """
    batch = openai_client.vector_stores.file_batches.create(
        vector_store_id=vector_store_id,
        file_ids=file_ids
    )
    print(f"✅ File batch created: {batch.id} (status={batch.status})")
    return batch


def poll_file_batch(vector_store_id: str, batch_id: str, poll_interval_sec: int = 2):
    """
    Poll the file batch until it reaches a terminal state.
    Terminal states: completed / failed / cancelled
    """
    terminal = {"completed", "failed", "cancelled"}

    while True:
        b = openai_client.vector_stores.file_batches.retrieve(
            vector_store_id=vector_store_id,
            batch_id=batch_id
        )

        if b.status in terminal:
            print(f"✅ Final batch status: {b.status}")
            # file_counts may include completed/failed/in_progress totals
            if hasattr(b, "file_counts"):
                print("file_counts:", b.file_counts)
            return b

        time.sleep(poll_interval_sec)


# ----------------------------
# 1) Create or reuse vector store
# ----------------------------
vector_store = create_or_reuse_vector_store(VECTOR_STORE_NAME, REUSE_VECTOR_STORE_ID)
VECTOR_STORE_ID = vector_store.id

# ----------------------------
# 2) Upload local artifacts as OpenAI Files
# ----------------------------
if "ALL_RAG_FILE_PATHS" not in globals() or not ALL_RAG_FILE_PATHS:
    raise ValueError(
        "ALL_RAG_FILE_PATHS is empty. Run Cell 2 first to export Drive PDFs and Notion RQ txt files."
    )

print(f"Total local artifacts to upload: {len(ALL_RAG_FILE_PATHS)}")
uploaded_file_ids = upload_files(ALL_RAG_FILE_PATHS)
print(f"✅ Uploaded {len(uploaded_file_ids)} files.")

# ----------------------------
# 3) Attach uploaded files to the vector store (file batch)
# ----------------------------
batch = attach_files_to_vector_store(VECTOR_STORE_ID, uploaded_file_ids)

# ----------------------------
# 4) Poll until indexing is complete
# ----------------------------
final_batch = poll_file_batch(VECTOR_STORE_ID, batch.id, poll_interval_sec=POLL_INTERVAL_SEC)

# Expose variables for downstream cells
VECTOR_STORE_FILE_IDS = uploaded_file_ids
print("\n✅ Vector store ready for retrieval.")
print("VECTOR_STORE_ID =", VECTOR_STORE_ID)


✅ Created vector store: vs_695b45e293f48191a78759668bb0cba8 (RQ-RAG-Drive+Notion)
Total local artifacts to upload: 64


Uploading files to OpenAI: 100%|████████████████| 64/64 [02:18<00:00,  2.17s/it]


✅ Uploaded 64 files.
✅ File batch created: vsfb_ibj_695b466e332881f4bc635011f77c33bd (status=in_progress)
✅ Final batch status: completed
file_counts: FileCounts(cancelled=0, completed=64, failed=0, in_progress=0, total=64)

✅ Vector store ready for retrieval.
VECTOR_STORE_ID = vs_695b45e293f48191a78759668bb0cba8


In [28]:
# ============================================================
# Cell 4 : Retrieval logic (top-k semantic search) [PATCHED]
# ============================================================

from typing import Any, Dict, List, Tuple, Optional
import textwrap

DEFAULT_TOP_K = int(globals().get("TOP_K_RETRIEVAL", 5))
DEFAULT_REWRITE_QUERY = True
DEFAULT_MAX_CHARS_PER_CHUNK = 800

def _as_dict(x: Any) -> Dict[str, Any]:
    """
    Convert SDK objects (pydantic-like) or dataclasses to dict safely.
    """
    if x is None:
        return {}
    if isinstance(x, dict):
        return x
    # pydantic v2
    if hasattr(x, "model_dump"):
        try:
            return x.model_dump()
        except Exception:
            pass
    # pydantic v1
    if hasattr(x, "dict"):
        try:
            return x.dict()
        except Exception:
            pass
    # generic fallback
    try:
        return dict(x)
    except Exception:
        pass
    try:
        return vars(x)
    except Exception:
        return {}

def _get_attr_or_key(obj: Any, key: str, default: Any = None) -> Any:
    """
    Read either obj.key (attribute) or obj[key] (dict).
    """
    if obj is None:
        return default
    if hasattr(obj, key):
        return getattr(obj, key, default)
    if isinstance(obj, dict):
        return obj.get(key, default)
    return default

def search_vector_store(
    query: str,
    top_k: int = DEFAULT_TOP_K,
    rewrite_query: bool = DEFAULT_REWRITE_QUERY,
    file_ids: Optional[List[str]] = None,
    filters: Optional[Dict[str, Any]] = None,
) -> Any:
    if not query or not isinstance(query, str):
        raise ValueError("query must be a non-empty string")

    kwargs = {
        "vector_store_id": VECTOR_STORE_ID,
        "query": query,
        "max_num_results": int(top_k),
        "rewrite_query": bool(rewrite_query),
    }
    if file_ids:
        kwargs["file_ids"] = file_ids
    if filters:
        kwargs["filters"] = filters

    return openai_client.vector_stores.search(**kwargs)

def _flatten_content_blocks(content: Any) -> str:
    """
    Flatten `content` blocks to plain text.
    Handles both dict blocks and SDK Content objects.
    Typical shapes:
      - [{"type":"text","text":"..."}]
      - [Content(type="text", text="..."), ...]
    """
    if not content:
        return ""

    parts = []
    # content might be a list-like of blocks
    for block in content:
        # block can be dict or object
        b_type = _get_attr_or_key(block, "type", None)
        if b_type == "text":
            txt = _get_attr_or_key(block, "text", "")
            if txt:
                parts.append(str(txt))
        else:
            # sometimes the text is nested; try dict conversion fallback
            bd = _as_dict(block)
            if bd.get("type") == "text" and bd.get("text"):
                parts.append(str(bd["text"]))

    return "\n".join(parts).strip()

def retrieve_topk_context(
    query: str,
    top_k: int = DEFAULT_TOP_K,
    rewrite_query: bool = DEFAULT_REWRITE_QUERY,
    max_chars_per_chunk: int = DEFAULT_MAX_CHARS_PER_CHUNK,
) -> Tuple[List[Dict[str, Any]], str]:
    res = search_vector_store(query=query, top_k=top_k, rewrite_query=rewrite_query)

    evidence: List[Dict[str, Any]] = []
    context_blocks: List[str] = []

    items = _get_attr_or_key(res, "data", []) or []
    for i, item in enumerate(items):
        filename = _get_attr_or_key(item, "filename", None)
        file_id = _get_attr_or_key(item, "file_id", None)
        score = _get_attr_or_key(item, "score", None)
        content = _get_attr_or_key(item, "content", None)
        attributes = _get_attr_or_key(item, "attributes", None)

        text = _flatten_content_blocks(content)
        if max_chars_per_chunk and len(text) > max_chars_per_chunk:
            text = text[:max_chars_per_chunk].rstrip() + "…"

        ev = {
            "rank": i + 1,
            "file_id": file_id,
            "filename": filename,
            "score": score,
            "text": text,
            "attributes": _as_dict(attributes) if attributes else {},
        }
        evidence.append(ev)

        context_blocks.append(
            f"[{i+1}] file={filename} (score={score})\n{text}"
        )

    context_str = "\n\n".join(context_blocks).strip()
    return evidence, context_str

def pretty_print_results(evidence: List[Dict[str, Any]]) -> None:
    if not evidence:
        print("No results.")
        return

    for ev in evidence:
        print("=" * 80)
        print(f"Rank: {ev['rank']} | Score: {ev.get('score')} | File: {ev.get('filename')}")
        if ev.get("attributes"):
            print("Attributes:", ev["attributes"])
        print("-" * 80)
        print(textwrap.fill(ev.get("text", ""), width=110))
    print("=" * 80)

# Optional quick test:
# evidence, context = retrieve_topk_context("VC syndication empirical strategy", top_k=5)
# pretty_print_results(evidence)
# print(context[:1200])


In [32]:
# ============================================================
# Cell 5 : Prompt Construction for Research Question Generation
# ============================================================
#
# This cell defines prompt templates and helper functions to construct
# an LLM-ready prompt for generating research questions (RQs) grounded
# in retrieved evidence from the vector store.
#
# Output of this cell:
# - `build_rq_messages(...)` : returns a list of messages (system + user)
# - `build_rq_input(...)`    : returns a single combined user prompt string (optional)
#
# Requirements (from previous cells):
# - retrieve_topk_context(...) (Cell 4) OR an equivalent `context_str`
# - LLM_MODEL, TEMPERATURE, MAX_TOKENS (Cell 1)
#
# Notes:
# - We keep the prompt modular to allow easy iteration on tone, format,
#   and constraints without touching downstream generation logic.
#

from typing import Dict, Any, List, Optional
import json
import textwrap


# ----------------------------
# Prompt Settings
# ----------------------------
N_RQ_CANDIDATES = 5          # how many RQs to generate per run
MAX_CONTEXT_CHARS = 8000     # safeguard to avoid overly large prompts
STYLE_CONSTRAINTS = [
    "Write ALL research questions and explanations in Japanese.",
    "Use clear, academic Japanese suitable for research planning.",
    "Do not mix English into the Japanese text, except for variable names or citations.",
    "Keep each research question specific, empirical, and testable.",
    "Avoid overly broad or purely descriptive questions.",
    "Ground each question in the provided evidence (cite evidence IDs).",
    "Prefer questions suitable for a PhD-level research agenda.",
]


# Optional: a simple rubric to score candidates
SCORING_RUBRIC = {
    "novelty": "How original / non-obvious is the question given the evidence?",
    "testability": "Can it be empirically tested with plausible data/identification?",
    "impact": "Does answering it matter for theory, policy, or practice?",
    "feasibility": "Is it feasible within a typical research timeline?",
}


# ----------------------------
# System Prompt
# ----------------------------
SYSTEM_PROMPT = """You are a careful research assistant helping a researcher develop empirical research questions.
You must ground proposals in the provided evidence snippets and avoid hallucinating citations.
If evidence is insufficient, propose a narrower question or explicitly request what additional evidence would be needed.
"""

# ----------------------------
# Output Schema (JSON-only)
# ----------------------------
# We ask the model to output strict JSON so the notebook can parse results reliably.

OUTPUT_SCHEMA_INSTRUCTIONS = """Return ONLY valid JSON (no markdown, no backticks).

Schema:
{
  "seed_idea": "<string>",
  "rq_candidates": [
    {
      "Name": "<日本語のResearch Question（1文）>",
      "Rationale / Background": "<日本語で意義（2-4文）>",
      "Gap Identified": "<日本語でギャップ/新規性（2-4文）>",
      "Proposed Approach": "<日本語で検証手法（2-5文）>",
      "Priority": "Medium",
      "Status": "New",
      "Tags": ["<english tag 1>", "<english tag 2>", "<english tag 3>", "<english tag 4>", "<english tag 5>"],
      "evidence_used": [1, 2]
    }
  ]
}

Rules:
- Write Name / Rationale / Gap / Proposed Approach in Japanese.
- Tags must be English, about 5 items, concise nouns (e.g., "VC", "Syndication", "Policy Shock").
- Priority must be exactly "Medium".
- Status must be exactly "New".
- evidence_used must reference evidence IDs in the provided context (e.g., [1], [2], ...).
- Keep each field concise. Avoid long lists or bullet points.
"""


# ----------------------------
# Helper: truncate context safely
# ----------------------------
def truncate_context(context_str: str, max_chars: int = MAX_CONTEXT_CHARS) -> str:
    if not context_str:
        return ""
    context_str = context_str.strip()
    if len(context_str) <= max_chars:
        return context_str
    return context_str[:max_chars].rstrip() + "\n…(truncated)…"


# ----------------------------
# Main builder: messages (system + user)
# ----------------------------
def build_rq_messages(
    seed_idea: str,
    context_str: str,
    n_candidates: int = N_RQ_CANDIDATES,
    style_constraints: Optional[List[str]] = None,
    scoring_rubric: Optional[Dict[str, str]] = None,
) -> List[Dict[str, str]]:
    """
    Construct chat messages for RQ generation.

    Args:
        seed_idea: The researcher's initial idea / interest statement.
        context_str: Retrieved evidence string (from Cell 4).
        n_candidates: Number of RQ candidates to generate.
        style_constraints: Optional list of style constraints.
        scoring_rubric: Optional rubric dict for scoring dimensions.

    Returns:
        messages: [{"role":"system","content":...}, {"role":"user","content":...}]
    """
    if not seed_idea or not isinstance(seed_idea, str):
        raise ValueError("seed_idea must be a non-empty string")

    style_constraints = style_constraints or STYLE_CONSTRAINTS
    scoring_rubric = scoring_rubric or SCORING_RUBRIC

    context_str = truncate_context(context_str)

    user_prompt = f"""
You will generate {n_candidates} candidate research questions based on the seed idea and the evidence.

Seed idea:
{seed_idea}

Style constraints:
- """ + "\n- ".join(style_constraints) + f"""

Scoring rubric (1-5 each):
{json.dumps(scoring_rubric, indent=2)}

Evidence context:
{context_str}

{OUTPUT_SCHEMA_INSTRUCTIONS}
""".strip()

    return [
        {"role": "system", "content": SYSTEM_PROMPT.strip()},
        {"role": "user", "content": user_prompt},
    ]


# ----------------------------
# Optional builder: single combined input string
# (Useful if you prefer a single `input=` field later.)
# ----------------------------
def build_rq_input(seed_idea: str, context_str: str) -> str:
    msgs = build_rq_messages(seed_idea=seed_idea, context_str=context_str)
    # join into a single string (system + user) for debugging/inspection
    return "SYSTEM:\n" + msgs[0]["content"] + "\n\nUSER:\n" + msgs[1]["content"]


# ----------------------------
# Quick preview (optional)
# ----------------------------
# seed_idea = "I want to study how VC syndication affects startup outcomes when policy shocks change investor constraints."
# evidence, context = retrieve_topk_context(seed_idea, top_k=5)
# print(build_rq_input(seed_idea, context)[:2000])


In [33]:
# ============================================================
# Cell 6 : LLM-based Question Generation using Retrieved Contexts (DB-aligned)
# ============================================================

from pydantic import BaseModel, Field
from typing import List, Literal
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime
import json

LLM_MODEL_STRUCTURED = "gpt-4o-mini"  # good default for structured parsing

# ----------------------------
# Pydantic schema aligned with your Notion DB
# ----------------------------
class RQCandidateNotion(BaseModel):
    Name: str
    Rationale___Background: str = Field(alias="Rationale / Background")
    Gap_Identified: str = Field(alias="Gap Identified")
    Proposed_Approach: str = Field(alias="Proposed Approach")
    Priority: Literal["Medium"] = "Medium"
    Status: Literal["New"] = "New"
    Tags: List[str] = Field(min_length=3, max_length=8)  # "about 5" → allow some flex
    evidence_used: List[int] = []

class RQOutputNotion(BaseModel):
    seed_idea: str
    rq_candidates: List[RQCandidateNotion]

# ----------------------------
# LLM call (structured parse)
# ----------------------------
def generate_rqs_structured(seed_idea: str, top_k: int, n_candidates: int, temperature: float):
    evidence, context_str = retrieve_topk_context(seed_idea, top_k=top_k)

    messages = build_rq_messages(
        seed_idea=seed_idea,
        context_str=context_str,
        n_candidates=n_candidates,
    )

    resp = openai_client.responses.parse(
        model=LLM_MODEL_STRUCTURED,
        input=messages,
        text_format=RQOutputNotion,
        temperature=temperature,
        max_output_tokens=MAX_TOKENS,  # MAX_TOKENSは 1500〜2500 推奨
    )

    parsed: RQOutputNotion = resp.output_parsed
    return evidence, context_str, parsed

# ----------------------------
# UI Widgets
# ----------------------------
seed_input = widgets.Textarea(
    value="",
    placeholder="研究アイデアを入力（2〜6文がおすすめ）...",
    description="Seed:",
    layout=widgets.Layout(width="900px", height="120px")
)

topk_slider = widgets.IntSlider(
    value=int(globals().get("TOP_K_RETRIEVAL", 5)),
    min=1, max=20, step=1,
    description="Top-k:",
    continuous_update=False
)

temp_slider = widgets.FloatSlider(
    value=float(globals().get("TEMPERATURE", 0.7)),
    min=0.0, max=1.2, step=0.1,
    description="Temp:",
    continuous_update=False
)

n_rq_slider = widgets.IntSlider(
    value=int(globals().get("N_RQ_CANDIDATES", 5)),
    min=3, max=12, step=1,
    description="#RQs:",
    continuous_update=False
)

generate_btn = widgets.Button(description="Retrieve + Generate (Notion format)", button_style="primary")

out_retrieval = widgets.Output()
out_generation = widgets.Output()
out_review = widgets.Output()

display(seed_input)
display(widgets.HBox([topk_slider, n_rq_slider, temp_slider, generate_btn]))
display(out_retrieval, out_generation, out_review)

# ----------------------------
# Review UI (Go/No-Go + manual edit)
# ----------------------------
_go_widgets = []
_edit_widgets = []

def build_review_ui(rq_candidates: List[dict]):
    _go_widgets.clear()
    _edit_widgets.clear()
    rows = []

    for i, rq in enumerate(rq_candidates):
        name = rq.get("Name", "")
        pr = rq.get("Priority", "")
        st = rq.get("Status", "")
        tags = rq.get("Tags", [])

        header = widgets.HTML(
            value=f"<b>RQ-{i+1:02d}</b> &nbsp; <span style='color:#666'>Priority={pr} | Status={st} | Tags={tags}</span><br/>{name}"
        )

        go = widgets.ToggleButtons(
            options=[("Go", True), ("No-Go", False)],
            value=False,
            description="Decision:"
        )

        edit = widgets.Textarea(
            value=json.dumps(rq, ensure_ascii=False, indent=2),
            description="Edit JSON:",
            layout=widgets.Layout(width="900px", height="220px")
        )

        _go_widgets.append(go)
        _edit_widgets.append(edit)
        rows.append(widgets.VBox([header, go, edit]))

    return widgets.VBox(rows)

def collect_selected_rqs() -> pd.DataFrame:
    selected = []
    for go_w, edit_w in zip(_go_widgets, _edit_widgets):
        if not go_w.value:
            continue
        selected.append(json.loads(edit_w.value))
    return pd.DataFrame(selected)

def on_generate_clicked(_):
    with out_retrieval:
        clear_output()
        print("Running retrieval...")

    with out_generation:
        clear_output()
        print("Calling the LLM (DB-aligned structured parse)...")

    with out_review:
        clear_output()

    seed_idea = seed_input.value.strip()
    if not seed_idea:
        with out_retrieval:
            clear_output()
            print("❌ Please enter a seed idea.")
        return

    try:
        evidence, context_str, parsed = generate_rqs_structured(
            seed_idea=seed_idea,
            top_k=int(topk_slider.value),
            n_candidates=int(n_rq_slider.value),
            temperature=float(temp_slider.value),
        )

        with out_retrieval:
            clear_output()
            print("✅ Retrieval completed.")
            pretty_print_results(evidence)

        # Convert parsed to list[dict] using aliases (original Notion-like keys)
        rqs_list = [c.model_dump(by_alias=True) for c in parsed.rq_candidates]

        with out_generation:
            clear_output()
            print("✅ Generation completed (parsed & validated).")
            print(f"Candidates: {len(rqs_list)}")
            print("Preview of first candidate:")
            print(json.dumps(rqs_list[0], ensure_ascii=False, indent=2)[:2000])

        review_ui = build_review_ui(rqs_list)
        save_btn = widgets.Button(description="Save Go-approved (as rq_selected_df)", button_style="success")
        save_out = widgets.Output()

        def on_save_clicked(__):
            with save_out:
                clear_output()
                df_sel = collect_selected_rqs()
                if df_sel.empty:
                    print("No Go-approved RQs selected.")
                    return

                global rq_selected_df, rq_df
                rq_df = pd.DataFrame(rqs_list)
                rq_selected_df = df_sel

                ts = datetime.now().strftime("%Y%m%d_%H%M%S")
                path = OUTPUT_DIR / f"rq_selected_{ts}.csv"
                df_sel.to_csv(path, index=False)
                print(f"✅ Saved {len(df_sel)} selected RQs to: {path}")
                display(df_sel)

        save_btn.on_click(on_save_clicked)

        with out_review:
            clear_output()
            display(review_ui)
            display(save_btn, save_out)

    except Exception as e:
        with out_generation:
            clear_output()
            print("❌ Error during generation:", type(e).__name__, str(e))

generate_btn.on_click(on_generate_clicked)


Textarea(value='', description='Seed:', layout=Layout(height='120px', width='900px'), placeholder='研究アイデアを入力（2…

Output()

Output()

Output()

In [34]:
# ============================================================
# Cell 7 : Post-processing (DB-aligned: Name/Tags/Priority/Status)
# ============================================================

import re
import json
from difflib import SequenceMatcher
from typing import Tuple

# ----------------------------
# Config
# ----------------------------
MIN_NAME_CHARS = 12
MAX_NAME_CHARS = 220
FUZZY_DUP_THRESHOLD = 0.92

REQUIRE_EVIDENCE = True
MIN_EVIDENCE_IDS = 1

# ----------------------------
# Helpers
# ----------------------------
def normalize_text(s: str) -> str:
    if s is None:
        return ""
    s = str(s).strip()
    s = re.sub(r"\s+", " ", s)
    s = s.replace("“", '"').replace("”", '"').replace("’", "'")
    s = re.sub(r"[。．\.]+$", "", s)
    return s

def similarity(a: str, b: str) -> float:
    return SequenceMatcher(None, a, b).ratio()

def safe_list(x):
    """Parse list-like fields that may be stored as JSON string."""
    if x is None:
        return []
    if isinstance(x, list):
        return x
    if isinstance(x, str):
        try:
            v = json.loads(x)
            return v if isinstance(v, list) else []
        except Exception:
            # fallback: comma split
            return [t.strip() for t in x.split(",") if t.strip()]
    return []

def safe_evidence_count(ev) -> int:
    ev_list = safe_list(ev)
    return len([x for x in ev_list if isinstance(x, int)])

def safe_tag_count(tags) -> int:
    tags_list = safe_list(tags)
    return len([x for x in tags_list if isinstance(x, str) and x.strip()])

def deduplicate_questions(df, col: str = "Name") -> Tuple[pd.DataFrame, pd.DataFrame]:
    if col not in df.columns:
        raise ValueError(f"Missing column: {col}")

    df = df.copy()
    df["_norm"] = df[col].map(normalize_text)

    # exact dedup
    df_exact = df.drop_duplicates(subset=["_norm"], keep="first").copy()
    removed_exact = df[~df.index.isin(df_exact.index)].copy()

    # fuzzy dedup (greedy)
    kept, removed = [], []
    norms = df_exact["_norm"].tolist()
    idxs = df_exact.index.tolist()

    for i, idx in enumerate(idxs):
        q = norms[i]
        is_dup = False
        for kept_idx in kept:
            q2 = df_exact.loc[kept_idx, "_norm"]
            if similarity(q, q2) >= FUZZY_DUP_THRESHOLD:
                is_dup = True
                removed.append(idx)
                break
        if not is_dup:
            kept.append(idx)

    df_dedup = df_exact.loc[kept].copy().reset_index(drop=True)
    removed_fuzzy = df_exact.loc[removed].copy().reset_index(drop=True)

    removed_all = pd.concat([removed_exact, removed_fuzzy], ignore_index=True)
    df_dedup = df_dedup.drop(columns=["_norm"])
    removed_all = removed_all.drop(columns=["_norm"], errors="ignore")
    return df_dedup, removed_all

def apply_basic_filters(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # Required columns for the DB-aligned pipeline
    required_cols = ["Name", "Rationale / Background", "Gap Identified", "Proposed Approach", "Priority", "Status", "Tags"]
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        raise ValueError(f"rq_df is not DB-aligned. Missing columns: {missing}")

    # normalize text fields
    for c in ["Name", "Rationale / Background", "Gap Identified", "Proposed Approach", "Priority", "Status"]:
        df[c] = df[c].map(normalize_text)

    # normalize list-like fields
    df["Tags"] = df["Tags"].map(safe_list)
    if "evidence_used" in df.columns:
        df["evidence_used"] = df["evidence_used"].map(safe_list)
    else:
        df["evidence_used"] = [[] for _ in range(len(df))]

    # length filter on Name
    df = df[df["Name"].str.len().between(MIN_NAME_CHARS, MAX_NAME_CHARS)]

    # evidence requirement
    df["evidence_count"] = df["evidence_used"].map(safe_evidence_count)
    if REQUIRE_EVIDENCE:
        df = df[df["evidence_count"] >= MIN_EVIDENCE_IDS]

    # tag sanity
    df["tag_count"] = df["Tags"].map(safe_tag_count)

    return df.reset_index(drop=True)

# ----------------------------
# Run post-processing for all candidates
# ----------------------------
if "rq_df" not in globals() or rq_df is None or len(rq_df) == 0:
    raise ValueError("rq_df is empty. Run Cell 6 first.")

rq_df_filtered = apply_basic_filters(rq_df)
rq_df_clean, rq_removed_df = deduplicate_questions(rq_df_filtered, col="Name")

# Ranking for triage:
# Priority/Status are fixed, so sort by evidence_count desc, then tag_count desc
rq_ranked_df = rq_df_clean.sort_values(
    by=["evidence_count", "tag_count", "Name"],
    ascending=[False, False, True],
).reset_index(drop=True)

print("✅ Post-processing complete.")
print(f"Candidates (raw): {len(rq_df)}")
print(f"After filters:    {len(rq_df_filtered)}")
print(f"After dedup:      {len(rq_df_clean)}")
print(f"Removed:          {len(rq_removed_df)}")

display(rq_ranked_df.head(10))

# ----------------------------
# Optional: post-process selected (Go-approved) candidates
# ----------------------------
if "rq_selected_df" in globals() and rq_selected_df is not None and len(rq_selected_df) > 0:
    rq_selected_filtered = apply_basic_filters(rq_selected_df)
    rq_selected_clean_df, rq_selected_removed_df = deduplicate_questions(rq_selected_filtered, col="Name")

    rq_selected_clean_df = rq_selected_clean_df.sort_values(
        by=["evidence_count", "tag_count", "Name"],
        ascending=[False, False, True],
    ).reset_index(drop=True)

    print("\n✅ Selected RQs post-processed.")
    print(f"Selected (raw):   {len(rq_selected_df)}")
    print(f"Selected (clean): {len(rq_selected_clean_df)}")

    display(rq_selected_clean_df)
else:
    rq_selected_clean_df = None
    print("\n(No Go-approved candidates found yet; skipping selected post-processing.)")


✅ Post-processing complete.
Candidates (raw): 5
After filters:    5
After dedup:      5
Removed:          0


,Name,Rationale / Background,Gap Identified,Proposed Approach,Priority,Status,Tags,evidence_used,evidence_count,tag_count
0,CVCとIVCの投資行動の違いが企業の学習成果に与える影響は何か？,CVCとIVCの投資行動には明確な違いがあり、それが企業の学習や成長にどのように影響するかを理解することは重要である。特に、学習の観点から両者の違いを明確にすることで、より効果的な投資戦略が見えてくる,CVCとIVCの投資行動の違いが企業の学習成果に与える影響に関する実証研究は限られており、そのメカニズムが不明である,CVCおよびIVCから資金を受けた企業のイノベーション成果を比較することで、両者の投資行動の違いが企業の学習成果に与える影響を分析する,Medium,New,"[CVC, IVC, Learning Outcomes, Investment Behavior, Innovation]","[1, 2]",2,5
1,CVCの実施が企業の長期的な成長に与える影響は何か？,CVCの役割は短期的な投資利益だけでなく、企業の長期的な成長にも影響を与える。本研究はそのメカニズムを明らかにすることで、CVCの価値を再評価することを目的とする,CVCの実施が企業の長期的な成長に与える影響についての実証研究は限られており、特にそのメカニズムが不明である,CVCを実施した企業の成長指標を追跡し、CVCの実施が長期的な成長にどのように寄与するかを分析する。定量的データと質的インタビューを併用する,Medium,New,"[CVC, Long-term Growth, Investment Impact, Qualitative Research, Quantitative Research]","[1, 2]",2,5
2,CVCの成長過程における学習効果は、ファイナンシャルリターン重視の姿勢によってどのように変化するか？,CVCは単なる資金提供に留まらず、企業の学習や成長に寄与する重要な要素である。本研究はファイナンシャルリターン重視がCVCの成長過程に与える影響を明らかにすることで、実務者にとっての示唆を提供することを目指す,従来の研究はCVCの学習過程におけるファイナンシャルリターンの影響を十分に考察しておらず、そのメカニズムが不明確である。特に、ファイナンシャルリターン重視が企業の探索行動に与える影響に関する実証研究は不足している,CVCを利用する企業を対象に、ファイナンシャルリターン重視度と学習成果の相関を多変量分析を用いて検証する。データとしては、CVC投資後のイノベーション成果や成長指標を用いる予定である,Medium,New,"[CVC, Learning, Financial Returns, Innovation, Growth]","[1, 2]",2,5
3,CVCの投資成功における制度環境の役割はどのようなものか？,制度環境はCVCの投資成功に影響を及ぼす重要な要因である。本研究は制度環境がCVCの投資成果に与える影響を明らかにすることで、政策提言に寄与することを目指す,制度環境がCVCの投資成功に与える影響についての実証研究が不足しており、特に制度の質と投資成果の関係が不明である,国や地域の制度環境指標を用いて、CVCの投資成功との関係を多変量回帰分析で検証する。制度環境の質を操作変数として考慮する,Medium,New,"[CVC, Institutional Environment, Investment Success, Policy Implications, Quantitative Analysis]","[1, 2]",2,5
4,CVCの投資成果におけるファイナンシャルリターン重視の影響を地域差で分析することは可能か？,地域によって起業環境や文化が異なるため、CVCの投資成果にも影響を与える。地域差を考慮することで、より具体的な戦略や政策提言が可能になる,CVCの投資成果と地域特性の関連性についての実証研究が不足しており、特にファイナンシャルリターン重視の影響を地域ごとに分析した研究は皆無である,地域別のCVC投資データを用いて、ファイナンシャルリターン重視の程度と投資成果との関係を多変量回帰分析で検証する。地域特性指標を交絡因子として考慮する,Medium,New,"[CVC, Regional Differences, Financial Returns, Investment Outcomes, Policy Analysis]","[1, 2]",2,5



✅ Selected RQs post-processed.
Selected (raw):   1
Selected (clean): 1


,Name,Rationale / Background,Gap Identified,Proposed Approach,Priority,Status,Tags,evidence_used,evidence_count,tag_count
0,CVCの成長過程における学習効果は、ファイナンシャルリターン重視の姿勢によってどのように変化するか？,CVCは単なる資金提供に留まらず、企業の学習や成長に寄与する重要な要素である。本研究はファイナンシャルリターン重視がCVCの成長過程に与える影響を明らかにすることで、実務者にとっての示唆を提供することを目指す,従来の研究はCVCの学習過程におけるファイナンシャルリターンの影響を十分に考察しておらず、そのメカニズムが不明確である。特に、ファイナンシャルリターン重視が企業の探索行動に与える影響に関する実証研究は不足している,CVCを利用する企業を対象に、ファイナンシャルリターン重視度と学習成果の相関を多変量分析を用いて検証する。データとしては、CVC投資後のイノベーション成果や成長指標を用いる予定である,Medium,New,"[CVC, Learning, Financial Returns, Innovation, Growth]","[1, 2]",2,5


In [36]:
# ============================================================
# Cell 8 : Export Results and Basic Inspection
# ============================================================
#
# This cell exports post-processed RQ candidates to disk and provides
# lightweight inspection helpers for quick review.
#
# Exports:
# - ranked candidates (rq_ranked_df)          -> CSV + JSON
# - removed/duplicate candidates (rq_removed_df) -> CSV
# - Go-approved candidates (rq_selected_clean_df, if available) -> CSV + JSON
#
# It also creates a small "run manifest" JSON file to make results traceable:
# - timestamp
# - vector_store_id
# - retrieval / generation settings
# - input source counts
#
# Requirements (from previous cells):
# - rq_ranked_df, rq_removed_df, rq_df_clean (Cell 7)
# - optionally rq_selected_clean_df (Cell 7)
# - OUTPUT_DIR (Cell 1)
#
import pandas as pd
import json
from datetime import datetime
from pathlib import Path

# ----------------------------
# Safety checks
# ----------------------------
if "rq_ranked_df" not in globals() or rq_ranked_df is None or len(rq_ranked_df) == 0:
    raise ValueError("rq_ranked_df is empty. Run Cell 6 and Cell 7 first.")

# ----------------------------
# Output paths
# ----------------------------
RUN_TS = datetime.now().strftime("%Y%m%d_%H%M%S")
EXPORT_DIR = OUTPUT_DIR / f"rq_run_{RUN_TS}"
EXPORT_DIR.mkdir(parents=True, exist_ok=True)

CANDIDATES_CSV = EXPORT_DIR / "rq_candidates_ranked.csv"
CANDIDATES_JSON = EXPORT_DIR / "rq_candidates_ranked.json"
REMOVED_CSV = EXPORT_DIR / "rq_candidates_removed.csv"

SELECTED_CSV = EXPORT_DIR / "rq_selected.csv"
SELECTED_JSON = EXPORT_DIR / "rq_selected.json"

MANIFEST_JSON = EXPORT_DIR / "run_manifest.json"

print(f"📦 Export directory: {EXPORT_DIR}")

# ----------------------------
# Helper: DataFrame -> JSON (records)
# ----------------------------
def df_to_records(df: pd.DataFrame) -> list:
    """
    Convert a DataFrame to JSON-serializable records.
    Safely handles numpy types.
    """
    records = json.loads(df.to_json(orient="records", force_ascii=False))
    return records

# ----------------------------
# Export: ranked candidates
# ----------------------------
rq_ranked_df.to_csv(CANDIDATES_CSV, index=False)
with open(CANDIDATES_JSON, "w", encoding="utf-8") as f:
    json.dump(df_to_records(rq_ranked_df), f, ensure_ascii=False, indent=2)

print(f"✅ Exported ranked candidates: {CANDIDATES_CSV.name}, {CANDIDATES_JSON.name}")

# ----------------------------
# Export: removed candidates
# ----------------------------
if "rq_removed_df" in globals() and rq_removed_df is not None and len(rq_removed_df) > 0:
    rq_removed_df.to_csv(REMOVED_CSV, index=False)
    print(f"✅ Exported removed candidates: {REMOVED_CSV.name}")
else:
    print("ℹ️ No removed candidates to export.")

# ----------------------------
# Export: selected candidates (if any)
# ----------------------------
if "rq_selected_clean_df" in globals() and rq_selected_clean_df is not None and len(rq_selected_clean_df) > 0:
    rq_selected_clean_df.to_csv(SELECTED_CSV, index=False)
    with open(SELECTED_JSON, "w", encoding="utf-8") as f:
        json.dump(df_to_records(rq_selected_clean_df), f, ensure_ascii=False, indent=2)
    print(f"✅ Exported selected candidates: {SELECTED_CSV.name}, {SELECTED_JSON.name}")
else:
    print("ℹ️ No selected candidates to export.")

# ----------------------------
# Run manifest (traceability)
# ----------------------------
manifest = {
    "run_timestamp": RUN_TS,
    "export_dir": str(EXPORT_DIR),
    "vector_store_id": globals().get("VECTOR_STORE_ID"),
    "llm_model": globals().get("LLM_MODEL_STRUCTURED", globals().get("LLM_MODEL", None)),
    "embedding_model": globals().get("EMBEDDING_MODEL", None),
    "top_k_retrieval": globals().get("TOP_K_RETRIEVAL", None),
    "min_similarity": globals().get("MIN_SIMILARITY", None),
    "temperature": globals().get("TEMPERATURE", None),
    "max_tokens": globals().get("MAX_TOKENS", None),
    "n_candidates_generated": int(len(rq_df)) if "rq_df" in globals() and rq_df is not None else None,
    "n_candidates_ranked": int(len(rq_ranked_df)),
    "n_candidates_selected": int(len(rq_selected_clean_df)) if "rq_selected_clean_df" in globals() and rq_selected_clean_df is not None else 0,
    "source_file_counts": {
        "drive_pdfs": int(len(globals().get("drive_downloaded_paths", []))),
        "notion_rq_txt": int(len(globals().get("notion_text_paths", []))),
        "all_rag_files": int(len(globals().get("ALL_RAG_FILE_PATHS", []))),
    },
}

with open(MANIFEST_JSON, "w", encoding="utf-8") as f:
    json.dump(manifest, f, ensure_ascii=False, indent=2)

print(f"🧾 Saved run manifest: {MANIFEST_JSON.name}")

# ----------------------------
# Basic inspection
# ----------------------------
print("\n--- Quick inspection ---")
display(rq_ranked_df.head(15))

if "rq_selected_clean_df" in globals() and rq_selected_clean_df is not None and len(rq_selected_clean_df) > 0:
    print("\n--- Selected (Go-approved) ---")
    display(rq_selected_clean_df)

print("\nDone.")

📦 Export directory: outputs/rq_run_20260105_151558
✅ Exported ranked candidates: rq_candidates_ranked.csv, rq_candidates_ranked.json
ℹ️ No removed candidates to export.
✅ Exported selected candidates: rq_selected.csv, rq_selected.json
🧾 Saved run manifest: run_manifest.json

--- Quick inspection ---


,Name,Rationale / Background,Gap Identified,Proposed Approach,Priority,Status,Tags,evidence_used,evidence_count,tag_count
0,CVCとIVCの投資行動の違いが企業の学習成果に与える影響は何か？,CVCとIVCの投資行動には明確な違いがあり、それが企業の学習や成長にどのように影響するかを理解することは重要である。特に、学習の観点から両者の違いを明確にすることで、より効果的な投資戦略が見えてくる,CVCとIVCの投資行動の違いが企業の学習成果に与える影響に関する実証研究は限られており、そのメカニズムが不明である,CVCおよびIVCから資金を受けた企業のイノベーション成果を比較することで、両者の投資行動の違いが企業の学習成果に与える影響を分析する,Medium,New,"[CVC, IVC, Learning Outcomes, Investment Behavior, Innovation]","[1, 2]",2,5
1,CVCの実施が企業の長期的な成長に与える影響は何か？,CVCの役割は短期的な投資利益だけでなく、企業の長期的な成長にも影響を与える。本研究はそのメカニズムを明らかにすることで、CVCの価値を再評価することを目的とする,CVCの実施が企業の長期的な成長に与える影響についての実証研究は限られており、特にそのメカニズムが不明である,CVCを実施した企業の成長指標を追跡し、CVCの実施が長期的な成長にどのように寄与するかを分析する。定量的データと質的インタビューを併用する,Medium,New,"[CVC, Long-term Growth, Investment Impact, Qualitative Research, Quantitative Research]","[1, 2]",2,5
2,CVCの成長過程における学習効果は、ファイナンシャルリターン重視の姿勢によってどのように変化するか？,CVCは単なる資金提供に留まらず、企業の学習や成長に寄与する重要な要素である。本研究はファイナンシャルリターン重視がCVCの成長過程に与える影響を明らかにすることで、実務者にとっての示唆を提供することを目指す,従来の研究はCVCの学習過程におけるファイナンシャルリターンの影響を十分に考察しておらず、そのメカニズムが不明確である。特に、ファイナンシャルリターン重視が企業の探索行動に与える影響に関する実証研究は不足している,CVCを利用する企業を対象に、ファイナンシャルリターン重視度と学習成果の相関を多変量分析を用いて検証する。データとしては、CVC投資後のイノベーション成果や成長指標を用いる予定である,Medium,New,"[CVC, Learning, Financial Returns, Innovation, Growth]","[1, 2]",2,5
3,CVCの投資成功における制度環境の役割はどのようなものか？,制度環境はCVCの投資成功に影響を及ぼす重要な要因である。本研究は制度環境がCVCの投資成果に与える影響を明らかにすることで、政策提言に寄与することを目指す,制度環境がCVCの投資成功に与える影響についての実証研究が不足しており、特に制度の質と投資成果の関係が不明である,国や地域の制度環境指標を用いて、CVCの投資成功との関係を多変量回帰分析で検証する。制度環境の質を操作変数として考慮する,Medium,New,"[CVC, Institutional Environment, Investment Success, Policy Implications, Quantitative Analysis]","[1, 2]",2,5
4,CVCの投資成果におけるファイナンシャルリターン重視の影響を地域差で分析することは可能か？,地域によって起業環境や文化が異なるため、CVCの投資成果にも影響を与える。地域差を考慮することで、より具体的な戦略や政策提言が可能になる,CVCの投資成果と地域特性の関連性についての実証研究が不足しており、特にファイナンシャルリターン重視の影響を地域ごとに分析した研究は皆無である,地域別のCVC投資データを用いて、ファイナンシャルリターン重視の程度と投資成果との関係を多変量回帰分析で検証する。地域特性指標を交絡因子として考慮する,Medium,New,"[CVC, Regional Differences, Financial Returns, Investment Outcomes, Policy Analysis]","[1, 2]",2,5



--- Selected (Go-approved) ---


,Name,Rationale / Background,Gap Identified,Proposed Approach,Priority,Status,Tags,evidence_used,evidence_count,tag_count
0,CVCの成長過程における学習効果は、ファイナンシャルリターン重視の姿勢によってどのように変化するか？,CVCは単なる資金提供に留まらず、企業の学習や成長に寄与する重要な要素である。本研究はファイナンシャルリターン重視がCVCの成長過程に与える影響を明らかにすることで、実務者にとっての示唆を提供することを目指す,従来の研究はCVCの学習過程におけるファイナンシャルリターンの影響を十分に考察しておらず、そのメカニズムが不明確である。特に、ファイナンシャルリターン重視が企業の探索行動に与える影響に関する実証研究は不足している,CVCを利用する企業を対象に、ファイナンシャルリターン重視度と学習成果の相関を多変量分析を用いて検証する。データとしては、CVC投資後のイノベーション成果や成長指標を用いる予定である,Medium,New,"[CVC, Learning, Financial Returns, Innovation, Growth]","[1, 2]",2,5



Done.


In [37]:
# ============================================================
# Cell 9 : Write Selected RQs to Notion (DB-aligned) [FULL VERSION]
# ============================================================
#
# This cell writes Go-approved (selected) RQs into your Notion database
# with the EXACT property structure:
# - Name (title)                 : Japanese Research Question
# - Rationale / Background       : Japanese
# - Gap Identified               : Japanese
# - Proposed Approach            : Japanese
# - Priority (select)            : "Medium" (fixed)
# - Status (status)              : "New" (fixed)
# - Tags (multi_select)          : ~5 English keywords
#
# It also performs:
# - DB schema validation (property existence + type match)
# - Option existence validation (Priority option "Medium", Status option "New")
# - Safe parsing for Tags (list or JSON string or comma-separated string)
#

import time
import json
import requests
import pandas as pd

NOTION_API_BASE = "https://api.notion.com/v1"
NOTION_WRITE_SLEEP_SEC = 0.35

PRIORITY_VALUE = "Medium"
STATUS_VALUE = "New"

# ----------------------------
# Safety checks
# ----------------------------
if "rq_selected_clean_df" not in globals() or rq_selected_clean_df is None or len(rq_selected_clean_df) == 0:
    raise ValueError("rq_selected_clean_df is empty. Please select (Go) some RQs first.")

if "NOTION_HEADERS" not in globals() or not isinstance(NOTION_HEADERS, dict):
    raise ValueError("NOTION_HEADERS is missing. Ensure Cell 1 (Notion config) ran successfully.")

if "NOTION_RQ_DB_ID" not in globals() or not NOTION_RQ_DB_ID:
    raise ValueError("NOTION_RQ_DB_ID is missing. Set it in env.txt and run Cell 1.")

# ----------------------------
# Notion property builders
# ----------------------------
def prop_title(text: str):
    text = "" if text is None else str(text)
    return {"title": [{"type": "text", "text": {"content": text}}]}

def prop_rich_text(text: str):
    text = "" if text is None else str(text)
    return {"rich_text": [{"type": "text", "text": {"content": text}}]} if text.strip() else {"rich_text": []}

def prop_select(name: str):
    name = "" if name is None else str(name).strip()
    return {"select": {"name": name}} if name else {"select": None}

def prop_status(name: str):
    name = "" if name is None else str(name).strip()
    return {"status": {"name": name}} if name else {"status": None}

def prop_multi_select(values):
    values = values or []
    return {"multi_select": [{"name": str(v).strip()} for v in values if str(v).strip()]}

# ----------------------------
# Notion API helpers
# ----------------------------
def notion_get_database(database_id: str):
    url = f"{NOTION_API_BASE}/databases/{database_id}"
    r = requests.get(url, headers=NOTION_HEADERS, timeout=60)
    r.raise_for_status()
    return r.json()

def notion_create_page(database_id: str, properties: dict):
    url = f"{NOTION_API_BASE}/pages"
    payload = {"parent": {"database_id": database_id}, "properties": properties}
    r = requests.post(url, headers=NOTION_HEADERS, json=payload, timeout=60)
    r.raise_for_status()
    return r.json()

def _option_names_for_property(prop_def: dict) -> list:
    """
    Extract option names for select/status properties.
    """
    t = prop_def.get("type")
    if t == "select":
        return [o.get("name") for o in (prop_def.get("select", {}).get("options", []) or [])]
    if t == "status":
        return [o.get("name") for o in (prop_def.get("status", {}).get("options", []) or [])]
    return []

def assert_notion_db_schema_and_options():
    """
    Validate:
    - required properties exist
    - required property types match
    - required select/status options exist
    """
    db = notion_get_database(NOTION_RQ_DB_ID)
    props = db.get("properties", {})

    required = {
        "Name": "title",
        "Rationale / Background": "rich_text",
        "Gap Identified": "rich_text",
        "Proposed Approach": "rich_text",
        "Priority": "select",
        "Status": "status",
        "Tags": "multi_select",
    }

    missing = [k for k in required.keys() if k not in props]
    if missing:
        raise ValueError(f"Notion DB is missing properties: {missing}")

    mismatched = []
    for k, expected_type in required.items():
        actual = props[k].get("type")
        if actual != expected_type:
            mismatched.append((k, expected_type, actual))
    if mismatched:
        raise ValueError(f"Notion DB property type mismatch: {mismatched}")

    # Validate options
    pr_options = _option_names_for_property(props["Priority"])
    if PRIORITY_VALUE not in pr_options:
        raise ValueError(
            f'Notion DB "Priority" select is missing option "{PRIORITY_VALUE}". '
            f"Available options: {pr_options}"
        )

    st_options = _option_names_for_property(props["Status"])
    if STATUS_VALUE not in st_options:
        raise ValueError(
            f'Notion DB "Status" status is missing option "{STATUS_VALUE}". '
            f"Available options: {st_options}"
        )

    print("✅ Notion DB schema + options look OK.")

# ----------------------------
# Tag parsing helper
# ----------------------------
def parse_tags(tags_value):
    """
    Accept:
      - list[str]
      - JSON string like '["VC","..."]'
      - comma-separated string like 'VC, Policy Shock, ...'
    Returns: list[str]
    """
    if tags_value is None:
        return []
    if isinstance(tags_value, list):
        return [str(t).strip() for t in tags_value if str(t).strip()]
    if isinstance(tags_value, str):
        s = tags_value.strip()
        if not s:
            return []
        # JSON list?
        try:
            v = json.loads(s)
            if isinstance(v, list):
                return [str(t).strip() for t in v if str(t).strip()]
        except Exception:
            pass
        # comma-separated
        return [t.strip() for t in s.split(",") if t.strip()]
    # fallback
    return [str(tags_value).strip()] if str(tags_value).strip() else []

# ----------------------------
# Validate Notion DB before writing
# ----------------------------
assert_notion_db_schema_and_options()

# ----------------------------
# Convert selected rows
# ----------------------------
rows = json.loads(rq_selected_clean_df.to_json(orient="records", force_ascii=False))

created = []
errors = []

for i, row in enumerate(rows, start=1):
    try:
        tags = parse_tags(row.get("Tags", []))

        # Build Notion properties (exact names must match DB)
        props = {
            "Name": prop_title(str(row.get("Name", ""))[:2000]),
            "Rationale / Background": prop_rich_text(row.get("Rationale / Background", "")),
            "Gap Identified": prop_rich_text(row.get("Gap Identified", "")),
            "Proposed Approach": prop_rich_text(row.get("Proposed Approach", "")),
            "Priority": prop_select(PRIORITY_VALUE),  # fixed
            "Status": prop_status(STATUS_VALUE),      # fixed
            "Tags": prop_multi_select(tags),
        }

        page = notion_create_page(NOTION_RQ_DB_ID, props)

        created.append({
            "idx": i,
            "Name": row.get("Name"),
            "notion_page_id": page.get("id"),
            "notion_url": page.get("url"),
        })

        time.sleep(NOTION_WRITE_SLEEP_SEC)

    except Exception as e:
        errors.append({
            "idx": i,
            "Name": row.get("Name"),
            "error": f"{type(e).__name__}: {str(e)}"
        })
        time.sleep(NOTION_WRITE_SLEEP_SEC)

print(f"✅ Created {len(created)} Notion pages.")
if errors:
    print(f"⚠️ Errors: {len(errors)} (showing up to 5)")
    for x in errors[:5]:
        print(x)

created_df = pd.DataFrame(created)
errors_df = pd.DataFrame(errors)

display(created_df.head(30))
if len(errors_df) > 0:
    display(errors_df.head(30))


✅ Notion DB schema + options look OK.
✅ Created 1 Notion pages.


,idx,Name,notion_page_id,notion_url
0,1,CVCの成長過程における学習効果は、ファイナンシャルリターン重視の姿勢によってどのように変化するか？,2df8e0e4-d162-81e3-8476-fb523bf51aed,https://www.notion.so/CVC-2df8e0e4d16281e38476fb523bf51aed
